# **zephyr-7B-beta-GGUF-infrence-on-collab**
Even though it is a quantized model, infrence on the colab CPU takes a few minutes.



---



* Download the specific quantized model from huggingface hub by using snapshot download

In [1]:
from huggingface_hub import snapshot_download

In [38]:
# model path from hugging face
model = "TheBloke/zephyr-7B-beta-GGUF"

In [8]:
# base path where you want to store
base_path ="./quantized_model"

In [13]:
# !rm -rf "quantized_model"

In [14]:
#  4 bit quantized model
# file_name ="zephyr-7b-beta.Q4_K_S.gguf"

In [35]:
# 2 bit quantized model
file_name2 ="zephyr-7b-beta.Q2_K.gguf"

In [15]:
# download model on local directory
# from huggingface_hub import hf_hub_download
# hf_hub_download(repo_id=model, filename=file_name,local_dir=base_path,local_dir_use_symlinks=False)

zephyr-7b-beta.Q4_K_S.gguf:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

'./quantized_model/zephyr-7b-beta.Q4_K_S.gguf'

In [39]:
# download model on local directory
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=model, filename=file_name2,local_dir=base_path,local_dir_use_symlinks=False)

zephyr-7b-beta.Q2_K.gguf:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

'./quantized_model/zephyr-7b-beta.Q2_K.gguf'

In [ ]:
# install all requirements from the file
!pip install -r requirements.txt

In [17]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma  # chroma for storing vector store locally
from langchain.embeddings import HuggingFaceBgeEmbeddings # for converting text to embedings
from langchain.document_loaders import PyPDFLoader # for dealing with pdf files


In [18]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

loader = PyPDFLoader("pet.pdf") # sample pdf
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

vector_store = Chroma.from_documents(texts, embeddings, collection_metadata={"hnsw:space": "cosine"}, persist_directory="stores/pet_cosine")

print("Vector Store Created.......")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Vector Store Created.......


In [24]:
print(texts[0].page_content)

Dogs are hugely popular pets. In fact, there are eight and a 
half million dogs being kept as pets in the UK alone. 
Known as ‘man’s best friend’, how should dogs be cared for and 
what do we know about them?
Diet
Dogs are omnivores, which means they need a well-balanced diet of meat and 
plant-based foods. They need one meal a day, unless the vet advises otherwise. 
Their teeth are well-developed, with sharp incisors for tearing meat, and molars 
for grinding plant foods.
They must have access to clean, fresh water at all times, or else they are at risk 
from becoming very poorly.
Environment
Dogs need a comfortable, draught-free, clean 
and quiet environment. They need to be able to 
sleep, undisturbed.
There needs to be access for them to go outside 
to the toilet regularly, and have some form of 
exercise appropriate for their breed, every day. 
After all, compare the type of walk a St Bernard 
needs compared to a dachshund!
Dogs need a place where they can go if they are


In [21]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import CTransformers
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from io import BytesIO
from langchain.document_loaders import PyPDFLoader



In [40]:
# local_llm = "quantized_model/zephyr-7b-beta.Q4_K_S.gguf"
local_llm = "/content/quantized_model/zephyr-7b-beta.Q2_K.gguf" # copy path of downloaded model

config = {
'max_new_tokens': 1024,
'repetition_penalty': 1.1,
'temperature': 0.1,
'top_k': 50,
'top_p': 0.9,
'stream': True,
'threads': int(os.cpu_count() / 2)
}

llm = CTransformers(
    model=local_llm,
    model_type="mistral",
    lib="avx2", #for CPU use
    **config
)

print("LLM Initialized...")

LLM Initialized...


In [26]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [41]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [42]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
load_vector_store = Chroma(persist_directory="stores/pet_cosine", embedding_function=embeddings)
retriever = load_vector_store.as_retriever(search_kwargs={"k":1})

In [43]:
query = "what is the fastest speed for a greyhound dog?"
semantic_search = retriever.get_relevant_documents(query)
print(semantic_search)

[Document(page_content='Dogs are hugely popular pets. In fact, there are eight and a \nhalf million dogs being kept as pets in the UK.\nFood and drink\nDogs need a good diet of meat and plant-based foods. They need \none meal a day. They have sharp teeth for tearing meat, and molars for \ngrinding.\nThey need fresh, clean water to keep them healthy.\nEnvironment\nDogs need a comfortable and quiet environment to sleep in peace.\nDogs need a place where they can go if they are frightened. Some dogs get more \nscared than others, and need somewhere to feel safe.\nDog behaviour\nDogs are clever and playful. They need to \nbe exercised and have toys to play with. \nDifferent breeds of dog have different skills, \nwhich you need to consider when looking \nafter them. Some will want longer walks, \nothers will prefer more time spent playing!Pet Care of a Dog\nDid You Know?\nThe fastest recorded speed \nfor a greyhound dog was \n42 miles per hour!\nPage 1 of 2', metadata={'page': 14, 'source':

In [44]:
print(semantic_search[0].page_content)

Dogs are hugely popular pets. In fact, there are eight and a 
half million dogs being kept as pets in the UK.
Food and drink
Dogs need a good diet of meat and plant-based foods. They need 
one meal a day. They have sharp teeth for tearing meat, and molars for 
grinding.
They need fresh, clean water to keep them healthy.
Environment
Dogs need a comfortable and quiet environment to sleep in peace.
Dogs need a place where they can go if they are frightened. Some dogs get more 
scared than others, and need somewhere to feel safe.
Dog behaviour
Dogs are clever and playful. They need to 
be exercised and have toys to play with. 
Different breeds of dog have different skills, 
which you need to consider when looking 
after them. Some will want longer walks, 
others will prefer more time spent playing!Pet Care of a Dog
Did You Know?
The fastest recorded speed 
for a greyhound dog was 
42 miles per hour!
Page 1 of 2


In [50]:
query = "what is the fastest speed for a greyhound dog?"

In [45]:
# print("######################################################################")
chain_type_kwargs = {"prompt": prompt}

In [46]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents = True,
    chain_type_kwargs= chain_type_kwargs,
    verbose=True
)

response = qa(query)

print(response)




> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'what is the fastest speed for a greyhound dog?', 'result': 'The fastest recorded speed for a greyhound dog is 42 miles per hour!', 'source_documents': [Document(page_content='Dogs are hugely popular pets. In fact, there are eight and a \nhalf million dogs being kept as pets in the UK.\nFood and drink\nDogs need a good diet of meat and plant-based foods. They need \none meal a day. They have sharp teeth for tearing meat, and molars for \ngrinding.\nThey need fresh, clean water to keep them healthy.\nEnvironment\nDogs need a comfortable and quiet environment to sleep in peace.\nDogs need a place where they can go if they are frightened. Some dogs get more \nscared than others, and need somewhere to feel safe.\nDog behaviour\nDogs are clever and playful. They need to \nbe exercised and have toys to play with. \nDifferent breeds of dog have different skills, \nwhich you need to consider when looking \nafter them. Some will

In [48]:
querys="Name two factors which might contribute to why some dogs might get scared?"
response = qa(querys)

print(response)



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'Name two factors which might contribute to why some dogs might get scared?', 'result': 'Dogs can get scared for different reasons, such as loud noises or sudden movements. Some breeds, like dachshunds, are also more sensitive than others, which might make them easier to scare compared to other breeds.', 'source_documents': [Document(page_content='needs compared to a dachshund!\nDogs need a place where they can go if they are \nfrightened. Dogs have very varied temperaments, \nor might have been previously mistreated, which means some scare more easily \nthan others.Pet Care of a Dog\nDid You Know?\nThe common ancestor \nof dogs is the wolf!\nPage 1 of 3', metadata={'page': 0, 'source': 'pet.pdf'})]}


* results are great but the infrence time is high
* try using other quantized models with the same approach

# ------------------------------------------------------------------------------**Thank You for tinkering**-------------------